In [4]:
from ultralytics import YOLO
import yaml
import os
from pathlib import Path
import torch

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [6]:
def setup_kaggle_paths():
    """
    Setup paths for Kaggle environment
    Returns base path for the dataset
    """
    # Typical Kaggle input path
    base_path = os.path.join('/kaggle', 'input', 'construction-site-safety-image-dataset-roboflow', 'css-data')
    
    # Verify the path exists
    if not os.path.exists(base_path):
        raise RuntimeError(f"Dataset not found at {base_path}. Please verify the dataset is loaded in Kaggle.")
    
    return base_path

def create_dataset_yaml(base_path):
    """
    Create YAML configuration file for YOLOv8 training with Kaggle paths
    """
    # Verify that required directories exist
    train_path = os.path.join(base_path, 'train', 'images')
    val_path = os.path.join(base_path, 'valid', 'images')
    test_path = os.path.join(base_path, 'test', 'images')
    
    # Check if directories exist
    for path in [train_path, val_path, test_path]:
        if not os.path.exists(path):
            raise RuntimeError(f"Required directory not found: {path}")
    
    dataset_config = {
        'path': base_path,  # dataset root dir
        'train': train_path,  # train images (absolute path)
        'val': val_path,  # val images (absolute path)
        'test': test_path,  # test images (absolute path)
        
        # Classes
        'names': {
            0: 'Hardhat',
            1: 'Mask',
            2: 'NO-Hardhat',
            3: 'NO-Mask',
            4: 'NO-Safety Vest',
            5: 'Person',
            6: 'Safety Cone',
            7: 'Safety Vest',
            8: 'machinery',
            9: 'vehicle'
        }
    }
    
    # Save YAML file to Kaggle working directory
    yaml_path = os.path.join('/kaggle', 'working', 'dataset.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(dataset_config, f, default_flow_style=False)
    
    return yaml_path

In [ ]:
def train_yolov8(data_yaml_path, epochs=100):
    """
    Train YOLOv8 model on PPE dataset
    """
    # Load a model
    model = YOLO('yolov8n.pt')  # load pretrained model (recommended for training)
    
    # Train the model
    results = model.train(
        data=data_yaml_path,
        epochs=epochs,
        imgsz=640,
        batch=16,
        name='ppe_detection',
        patience=20,  # Early stopping patience
        save=True,  # Save best model
        device='0' if torch.cuda.is_available() else 'cpu'  # Use GPU if available
    )
    
    return results, model

def validate_model(model):
    """
    Validate the trained model
    """
    # Validate the model
    metrics = model.val()
    return metrics

def predict_ppe(model, image_path):
    """
    Make predictions on a single image
    """
    results = model.predict(
        source=image_path,
        conf=0.25,  # Confidence threshold
        iou=0.45,   # NMS IoU threshold
        save=True   # Save results
    )
    return results

In [ ]:
def main():
    # Set up paths
    data_dir = '/kaggle/input/construction-site-safety-image-dataset-roboflow/css-data'  # Replace with your dataset path
    
    # Create dataset YAML
    yaml_path = create_dataset_yaml(data_dir)
    
    # Train model
    print("Starting training...")
    results, model = train_yolov8(yaml_path)
    
    # Validate model
    print("Validating model...")
    metrics = validate_model(model)
    
    print("Training completed!")
    print(f"Best mAP: {metrics.box.map}")  # Print mean Average Precision
    
    # Example prediction
    test_image = '/kaggle/input/construction-site-safety-image-dataset-roboflow/css-data/test/images/003357_jpg.rf.9867f91e88089bb68dc95947d5116d14.jpg'
    if os.path.exists(test_image):
        results = predict_ppe(model, test_image)
        print("Prediction completed and saved!")

if __name__ == "__main__":
    main()

In [ ]:
from ultralytics import YOLO
import shutil
import os

def save_model_locally():
    # Path to your trained model in Kaggle
    kaggle_model_path = '/kaggle/working/runs/detect/ppe_detection2/weights/best.pt'
    
    # Local path where you want to save the model
    local_save_path = 'ppe_detection_model.pt'
    
    # Copy the model file
    shutil.copy(kaggle_model_path, local_save_path)
    print(f"Model saved locally at: {local_save_path}")

if __name__ == "__main__":
    save_model_locally()